In [1]:
import pandas as pd
import numpy as np
import re

# Overview
- chill dashboard for football data
- so far will analyse players and their market values
- data source: https://www.kaggle.com/datasets/davidcariboo/player-scores?resource=download&select=competitions.csv

In [2]:
competitions = pd.read_csv("data/competitions.csv")
competitions.head()

,competition_id,competition_code,name,sub_type,type,country_id,country_name,domestic_league_code,confederation,url,is_major_national_league
0,CIT,italy-cup,italy-cup,domestic_cup,domestic_cup,75,Italy,IT1,europa,https://www.transfermarkt.co.uk/italy-cup/star...,False
1,NLSC,johan-cruijff-schaal,johan-cruijff-schaal,domestic_super_cup,other,122,Netherlands,NL1,europa,https://www.transfermarkt.co.uk/johan-cruijff-...,False
2,GRP,kypello-elladas,kypello-elladas,domestic_cup,domestic_cup,56,Greece,GR1,europa,https://www.transfermarkt.co.uk/kypello-ellada...,False
3,POSU,supertaca-candido-de-oliveira,supertaca-candido-de-oliveira,domestic_super_cup,other,136,Portugal,PO1,europa,https://www.transfermarkt.co.uk/supertaca-cand...,False
4,RUSS,russian-super-cup,russian-super-cup,domestic_super_cup,other,141,Russia,RU1,europa,https://www.transfermarkt.co.uk/russian-super-...,False


In [3]:
interest = competitions[competitions.is_major_national_league==True].competition_id.unique().tolist()

In [4]:
clubs = pd.read_csv("data/clubs.csv")
clubs_of_int = clubs[clubs.domestic_competition_id.isin(interest)]
clubs_of_int.info()

<class 'pandas.core.frame.DataFrame'>
Index: 173 entries, 0 to 438
Data columns (total 17 columns):
 #   Column                   Non-Null Count  Dtype  
---  ------                   --------------  -----  
 0   club_id                  173 non-null    int64  
 1   club_code                173 non-null    object 
 2   name                     173 non-null    object 
 3   domestic_competition_id  173 non-null    object 
 4   total_market_value       0 non-null      float64
 5   squad_size               173 non-null    int64  
 6   average_age              172 non-null    float64
 7   foreigners_number        173 non-null    int64  
 8   foreigners_percentage    172 non-null    float64
 9   national_team_players    173 non-null    int64  
 10  stadium_name             173 non-null    object 
 11  stadium_seats            173 non-null    int64  
 12  net_transfer_record      173 non-null    object 
 13  coach_name               0 non-null      float64
 14  last_season              173 no

In [17]:
competitions[competitions.is_major_national_league==True]

,competition_id,competition_code,name,sub_type,type,country_id,country_name,domestic_league_code,confederation,url,is_major_national_league
9,ES1,laliga,laliga,first_tier,domestic_league,157,Spain,ES1,europa,https://www.transfermarkt.co.uk/laliga/startse...,True
10,FR1,ligue-1,ligue-1,first_tier,domestic_league,50,France,FR1,europa,https://www.transfermarkt.co.uk/ligue-1/starts...,True
11,IT1,serie-a,serie-a,first_tier,domestic_league,75,Italy,IT1,europa,https://www.transfermarkt.co.uk/serie-a/starts...,True
16,GB1,premier-league,premier-league,first_tier,domestic_league,189,England,GB1,europa,https://www.transfermarkt.co.uk/premier-league...,True
33,L1,bundesliga,bundesliga,first_tier,domestic_league,40,Germany,L1,europa,https://www.transfermarkt.co.uk/bundesliga/sta...,True


In [5]:
sel = ['competition_id', 'competition_code', 'country_name']
clubs_of_int = clubs_of_int.merge(competitions[sel], how='left', left_on='domestic_competition_id', right_on='competition_id')

In [6]:
del competitions, clubs, interest

In [7]:
players = pd.read_csv("data/players.csv")
players.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 32601 entries, 0 to 32600
Data columns (total 23 columns):
 #   Column                                Non-Null Count  Dtype  
---  ------                                --------------  -----  
 0   player_id                             32601 non-null  int64  
 1   first_name                            30539 non-null  object 
 2   last_name                             32601 non-null  object 
 3   name                                  32601 non-null  object 
 4   last_season                           32601 non-null  int64  
 5   current_club_id                       32601 non-null  int64  
 6   player_code                           32601 non-null  object 
 7   country_of_birth                      29802 non-null  object 
 8   city_of_birth                         30146 non-null  object 
 9   country_of_citizenship                32218 non-null  object 
 10  date_of_birth                         32554 non-null  object 
 11  sub_position   

In [8]:
interest = clubs_of_int.club_id.tolist()
players = players[(players.current_club_id.isin(interest))]

## Create the main table to analyse
1. clubs_of_int
2. players

- players (current_club_id) left join clubs_of_int (club_id)

In [9]:
# cleaning on the clubs data
print(clubs_of_int.columns)
clubs_of_int.sample(5)

Index(['club_id', 'club_code', 'name', 'domestic_competition_id',
       'total_market_value', 'squad_size', 'average_age', 'foreigners_number',
       'foreigners_percentage', 'national_team_players', 'stadium_name',
       'stadium_seats', 'net_transfer_record', 'coach_name', 'last_season',
       'filename', 'url', 'competition_id', 'competition_code',
       'country_name'],
      dtype='object')


,club_id,club_code,name,domestic_competition_id,total_market_value,squad_size,average_age,foreigners_number,foreigners_percentage,national_team_players,stadium_name,stadium_seats,net_transfer_record,coach_name,last_season,filename,url,competition_id,competition_code,country_name
16,4,1-fc-nurnberg,1.FC Nuremberg,L1,NaN,33,25.2,7,21.2,2,Max-Morlock-Stadion,50000,+€2.63m,NaN,2018,../data/raw/transfermarkt-scraper/2018/clubs.j...,https://www.transfermarkt.co.uk/1-fc-nurnberg/...,L1,bundesliga,Germany
6,603,cardiff-city,Cardiff City,GB1,NaN,28,25.5,24,85.7,8,Cardiff City Stadium,33280,€-655k,NaN,2018,../data/raw/transfermarkt-scraper/2018/clubs.j...,https://www.transfermarkt.co.uk/cardiff-city/s...,GB1,premier-league,England
120,3,1-fc-koln,1.FC Köln,L1,NaN,34,24.6,11,32.4,6,RheinEnergieSTADION,50000,+€1.60m,NaN,2023,../data/raw/transfermarkt-scraper/2023/clubs.j...,https://www.transfermarkt.co.uk/1-fc-koln/star...,L1,bundesliga,Germany
25,2036,1-fc-heidenheim-1846,1. Fußballclub Heidenheim 1846,L1,NaN,28,26.9,4,14.3,1,Voith-Arena,15000,+€7.45m,NaN,2024,../data/raw/transfermarkt-scraper/2024/clubs.j...,https://www.transfermarkt.co.uk/1-fc-heidenhei...,L1,bundesliga,Germany
149,1210,as-livorno,US Livorno 1915,IT1,NaN,29,24.1,5,17.2,0,Armando Picchi,19238,+-0,NaN,2013,../data/raw/transfermarkt-scraper/2013/clubs.j...,https://www.transfermarkt.co.uk/as-livorno/sta...,IT1,serie-a,Italy


In [9]:
club_join = [
    'club_id', 'country_name'
]
players = players.merge(clubs_of_int[club_join], how='left', left_on='current_club_id', right_on='club_id').drop(['club_id'], axis=1)

In [10]:
players = players.rename({'country_name': 'country_league'}, axis=1)

In [11]:
cur_players = players[players.last_season==2024]
team_mv = cur_players.groupby(['current_club_id'])['market_value_in_eur'].sum()
team_mv = pd.DataFrame(team_mv).reset_index().rename({'market_value_in_eur': 'total_market_value'}, axis=1)

club_drop = [
    'total_market_value', 'coach_name'
]

clubs_of_int = clubs_of_int.drop(club_drop, axis=1)
clubs_of_int = clubs_of_int.merge(team_mv, how='left', left_on='club_id', right_on='current_club_id').drop(['current_club_id'], axis=1)

In [ ]:
# Extract Net Transfer info
clubs_of_int.loc[:, 'net_transfer_sign'] = clubs_of_int.net_transfer_record.str.extract(r"([+-])", expand=False)
clubs_of_int.loc[:, 'net_transfer_value'] = clubs_of_int.net_transfer_record.str.extract(r"([0-9]+(?:\.[0-9]+)?)", expand=False).astype(float)

In [22]:
def calculate_net_transfer(row):
    if row['net_transfer_sign'] == "-":
        return row['net_transfer_value'] * -1.0
    else:
        return row['net_transfer_value'] * 1.0

clubs_of_int.loc[:, 'net_transfer_Mil'] = clubs_of_int.apply(calculate_net_transfer, axis=1)

# Analysis: Players

In [11]:
# 11,698 for players in Big 5 Leagues (All)
players.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 11698 entries, 0 to 11697
Data columns (total 24 columns):
 #   Column                                Non-Null Count  Dtype  
---  ------                                --------------  -----  
 0   player_id                             11698 non-null  int64  
 1   first_name                            11252 non-null  object 
 2   last_name                             11698 non-null  object 
 3   name                                  11698 non-null  object 
 4   last_season                           11698 non-null  int64  
 5   current_club_id                       11698 non-null  int64  
 6   player_code                           11698 non-null  object 
 7   country_of_birth                      11207 non-null  object 
 8   city_of_birth                         11320 non-null  object 
 9   country_of_citizenship                11573 non-null  object 
 10  date_of_birth                         11675 non-null  object 
 11  sub_position   

The data seems to be quite complete. Just a few things to check:
1. missing market_value
2. height_in_cm

**Conclusion**
<br>
It's possible not to have these data, just meaning this player is too rare to be rated. :)

In [ ]:
players[players.market_value_in_eur.isna()]
players[players.height_in_cm.isna()]

,player_id,first_name,last_name,name,last_season,current_club_id,player_code,country_of_birth,city_of_birth,country_of_citizenship,...,foot,height_in_cm,contract_expiration_date,agent_name,image_url,url,current_club_domestic_competition_id,current_club_name,market_value_in_eur,highest_market_value_in_eur
6946,85316,Kilian,Falcón,Kilian Falcón,2012,142,kilian-falcon,Spain,Las Palmas de Gran Canaria,Spain,...,NaN,184.0,NaN,NaN,https://img.a.transfermarkt.technology/portrai...,https://www.transfermarkt.co.uk/kilian-falcon/...,ES1,Real Zaragoza,NaN,NaN
6949,85326,Yeray,Gómez,Yeray Gómez,2014,3302,yeray-gomez,Spain,Selva,Spain,...,NaN,192.0,NaN,NaN,https://img.a.transfermarkt.technology/portrai...,https://www.transfermarkt.co.uk/yeray-gomez/pr...,ES1,UD Almería,NaN,NaN
8710,121232,Callum,Driver,Callum Driver,2013,379,callum-driver,England,London,England,...,right,173.0,NaN,NaN,https://img.a.transfermarkt.technology/portrai...,https://www.transfermarkt.co.uk/callum-driver/...,GB1,West Ham United Football Club,NaN,NaN
9029,126555,Victor,Fuchs,Victor Fuchs,2012,681,victor-fuchs,France,Arcachon,France,...,right,174.0,NaN,NaN,https://img.a.transfermarkt.technology/portrai...,https://www.transfermarkt.co.uk/victor-fuchs/p...,ES1,Real Sociedad de Fútbol S.A.D.,NaN,NaN
9108,127185,Kévin,Crépel,Kévin Crépel,2012,1159,kevin-crepel,France,Armentières,France,...,right,195.0,NaN,NaN,https://img.a.transfermarkt.technology/portrai...,https://www.transfermarkt.co.uk/kevin-crepel/p...,FR1,AS Nancy-Lorraine,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
32499,1210422,Aboubacar,Bassinga,Aboubacar Bassinga,2023,472,aboubacar-bassinga,NaN,NaN,Cote d'Ivoire,...,NaN,NaN,NaN,Gesport Espizua SL,https://img.a.transfermarkt.technology/portrai...,https://www.transfermarkt.co.uk/aboubacar-bass...,ES1,Unión Deportiva Las Palmas S.A.D.,NaN,NaN
32504,1222050,Rocco,Di Vico,Rocco Di Vico,2023,380,rocco-di-vico,NaN,NaN,Italy,...,NaN,NaN,NaN,NaN,https://img.a.transfermarkt.technology/portrai...,https://www.transfermarkt.co.uk/rocco-di-vico/...,IT1,US Salernitana 1919,NaN,NaN
32507,1225825,Christian,Chigozie,Christian Chigozie,2023,1031,christian-chigozie,NaN,NaN,England,...,NaN,NaN,NaN,NaN,https://img.a.transfermarkt.technology/portrai...,https://www.transfermarkt.co.uk/christian-chig...,GB1,Luton Town,NaN,NaN
32521,1237073,Lander,Emery,Lander Emery,2023,405,lander-emery,NaN,NaN,Spain,...,NaN,NaN,NaN,MYSPORTDEAL,https://img.a.transfermarkt.technology/portrai...,https://www.transfermarkt.co.uk/lander-emery/p...,GB1,Aston Villa Football Club,NaN,NaN


In [12]:
cur_players = players[players.last_season==2024]

In [13]:
cur_players.groupby(['country_league'])['current_club_id'].nunique()

country_league
England    20
France     18
Germany    18
Italy      20
Spain      20
Name: current_club_id, dtype: int64

In [14]:
cur_players.groupby(['country_league'])['player_id'].nunique()

country_league
England    540
France     473
Germany    507
Italy      568
Spain      493
Name: player_id, dtype: int64

In [15]:
cur_players.loc[:, 'playing_home'] = cur_players.country_of_citizenship==cur_players.country_league

C:\Users\kqtan\AppData\Local\Temp\ipykernel_25528\937123509.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  cur_players.loc[:, 'playing_home'] = cur_players.country_of_citizenship==cur_players.country_league


In [16]:
cur_players['foot'].value_counts()

foot
right    1803
left      671
both       81
Name: count, dtype: int64

# Analysis: Clubs

In [18]:
team_mv = cur_players.groupby(['current_club_id'])['market_value_in_eur'].sum()
team_mv = pd.DataFrame(team_mv).reset_index()

In [ ]:
club_drop = [
    'total_market_value', 'coach_name'
]

team_mv = team_mv.rename({'market_value_in_eur': 'total_market_value'}, axis=1)
clubs_of_int = clubs_of_int.drop(club_drop, axis=1)
clubs_of_int = clubs_of_int.merge(team_mv, how='left', left_on='club_id', right_on='current_club_id').drop(['current_club_id'], axis=1)

In [23]:
clubs_of_int.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 173 entries, 0 to 172
Data columns (total 22 columns):
 #   Column                   Non-Null Count  Dtype  
---  ------                   --------------  -----  
 0   club_id                  173 non-null    int64  
 1   club_code                173 non-null    object 
 2   name                     173 non-null    object 
 3   domestic_competition_id  173 non-null    object 
 4   squad_size               173 non-null    int64  
 5   average_age              172 non-null    float64
 6   foreigners_number        173 non-null    int64  
 7   foreigners_percentage    172 non-null    float64
 8   national_team_players    173 non-null    int64  
 9   stadium_name             173 non-null    object 
 10  stadium_seats            173 non-null    int64  
 11  net_transfer_record      173 non-null    object 
 12  last_season              173 non-null    int64  
 13  filename                 173 non-null    object 
 14  url                      1

In [29]:
clubs_of_int.to_parquet('cleaned_data/t5_league_clubs.parquet', index=False)